In [1]:
import sys
import numpy as np
import tqdm

sys.path.append("../")
from lwreg import standardization_lib
import lwreg
from rdkit import Chem

import yaml
solvent_dict = yaml.load(open("../Simulation/solvents.yml"), Loader=yaml.FullLoader)[
    "solvent_mapping_dict"
]

with open("/home/kpaul/.lwreg_key", "r") as f:
    key = f.read().strip()

config = lwreg.utils.defaultConfig()
config["standardization"] = standardization_lib.NoStandardization()
config["dbname"] = "solvent_forces"
config["dbtype"] = "postgresql"
config["removeHs"] = 0
config["registerConformers"] = True
config["hashConformer"] = 0  # set to 0
config["numConformerDigits"] = 3  #
config["host"] = "scotland"
config["user"] = "kpaul_lwreg"
config["password"] = key

In [2]:
save_folder = "/fileserver/pine/pine8/kpaul/multi_solvent_pub/data/Extracted_forces_SDFs/"
for solvent in tqdm.tqdm(solvent_dict.keys()):
    print(f"Processing {solvent}")
    if solvent == "tip3p": # TIP3P is already published
        print("Skipping TIP3P")
        continue

    sdf_filename = f"{save_folder}dataset_{solvent}.sdf"

    cn = lwreg.utils.connect(config)
    curs = cn.cursor()
    curs.execute(
        "select canonical_smiles, hashes.molregno, calcs.conf_id, calcs.positions, calcs.forces, calcs.usage_flag from solvent_%s.explicit_calculations calcs join conformers using (conf_id) join hashes using (molregno) where usage_flag='train' or usage_flag='test'" %solvent
    )
    data_list = []

    with Chem.SDWriter(sdf_filename) as w:

        for smiles, molregno, conf_id, positions, forces, usage_flag in tqdm.tqdm(curs):

            results = lwreg.retrieve(config,id=(molregno,conf_id),as_submitted=True)
            mol = Chem.MolFromMolBlock(results[(molregno,conf_id)][0],sanitize=False)
            diff = mol.GetConformer().GetPositions() - positions
            assert np.isclose(diff,0.0,atol=1e-5).all()
            mol.SetProp("forces",np.array2string(np.array(forces)[:,1:], separator=","))
            mol.SetProp("usage_flag",usage_flag)
            mol.SetProp("SMILES",smiles)
            
            w.write(mol)
    cn.commit()

  0%|          | 0/39 [00:00<?, ?it/s]

Processing tip3p
Skipping TIP3P
Processing Chloroform


1089696it [47:10, 384.96it/s]
  5%|▌         | 2/39 [56:02<17:16:54, 1681.47s/it]

Processing Methanol


1107512it [48:00, 384.49it/s]
  8%|▊         | 3/39 [1:54:37<24:27:16, 2445.46s/it]

...

366828it [17:33, 348.05it/s]
 95%|█████████▍| 37/39 [13:58:49<40:45, 1222.97s/it]  

Processing Benzonitrile


367008it [17:33, 348.45it/s]
 97%|█████████▋| 38/39 [14:19:23<20:26, 1226.31s/it]

Processing oxylol


358606it [17:02, 350.65it/s]
100%|██████████| 39/39 [14:39:22<00:00, 1352.88s/it]
